In [80]:
%reset -f
import glob, os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nms import non_max_suppression_fast

In [81]:
basedir = "/home/kalfasyan/data/images/sticky_plates/2020"
date = '20200723'
images_path = f"{basedir}/{date}/plates"
imgpaths = glob.glob(f"{images_path}/**jpg")

# imgpath

In [82]:
def overlay_image_nms(imgpath, plot_orig=True):
    image = cv2.imread(imgpath)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    gray = image[:,:,2] #cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    filtered = cv2.medianBlur(gray, 5) # 13
    # (thresh, blackAndWhiteImage) = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY)
    edged = cv2.Canny(filtered, 50, 200)

    # Highlighting detections
    (cnts,_) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edged_image = image.copy()
    cv2.drawContours(edged_image, cnts, -1, (0, 255, 0), 1);

    coordinates, contours = [], []
    for cnt in cnts:
        M = cv2.moments(cnt)
        try:
            cx = int(M['m10']/M['m00'])
        except:
            cx = np.nan
        try:
            cy = int(M['m01']/M['m00'])
        except:
            cy = np.nan
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)

        if (12 < area) and (50 < perimeter):
            if np.nan not in [cx, cy]:
                x,y,w,h = cv2.boundingRect(cnt)
                edged_image = cv2.rectangle(edged_image, (x,y), (x+w, y+h), (0,255,0), 2)
                coordinates.append((x,y,x+w, y+h))
                contours.append(cnt)

    # load the image and clone it
    boundingBoxes = np.array(coordinates)

    img = cv2.imread(imgpath)
#     cv2.imshow('IMAGE1',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    platename = imgpath.split('/')[-1][:-4]
    all_bbox_features = []
    bbox_features = {}

    # perform non-maximum suppression on the bounding boxes
    pick, idxs = non_max_suppression_fast(boundingBoxes, 0.15)
    # loop over the picked bounding boxes and draw them
    for p, (startX, startY, endX, endY) in enumerate(pick):
        cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)
    #     bbox = img[startY:endY+1, startX:endX+1]
        idx = np.where(np.all(pick[p]==boundingBoxes,axis=1))[0][0]
        cnt = contours[idx]
        bbox_features['plate_name'] = platename
        bbox_features['index'] = p+1
        bbox_features['area'] = cv2.contourArea(cnt)
        bbox_features['perimeter'] = cv2.arcLength(cnt, True)
    #     bbox_features['ellipse'] = cv2.fitEllipse(cnt)
        bbox_features['convexity_bool'] = cv2.isContourConvex(cnt)
    #     bbox_features['hull'] =  cv2.convexHull(cnt)
        _, bbox_features['enclosing_circle_radius'] = cv2.minEnclosingCircle(cnt)
        bbox_features['R'] = img[startY:endY+1, startX:endX+1, 0].mean()
        bbox_features['G'] = img[startY:endY+1, startX:endX+1, 1].mean()
        bbox_features['B'] = img[startY:endY+1, startX:endX+1, 2].mean()
        bbox_features['startX'] = startX
        bbox_features['startY'] = startY
        bbox_features['endX'] = endX+1
        bbox_features['endY'] = endY+1
        bbox_features['Class'] = ''
        all_bbox_features.append(pd.Series(bbox_features))



    for i, c in enumerate(pick):
        cv2.putText(img, "{}".format(i + 1), (c[0]-20, c[1]-10), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0,0,250), 4)

    cv2.putText(img, platename, (10, 4150), cv2.FONT_HERSHEY_DUPLEX, 5., (0,0,250), 5)

    if plot_orig:
        cv2.imshow('IMAGE',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    cv2.imwrite(f"{imgpath[:-4]}_overlay.jpg",img)

    print(f"{platename}")
    print(f"{len(coordinates)} insects counted in captured image.")
    print(f"{len(pick)} images after non-maximal-suppression\n")

    df = pd.concat(all_bbox_features,axis=1).T
    df.to_csv(f"{imgpath[:-4]}.txt", sep=',')
    
    return img, df

In [83]:
all_dfs = []
for imgpath in imgpaths:
    resimg, df = overlay_image_nms(imgpath, plot_orig=False)
    all_dfs.append(df)

Racour_20200702_B_F10_51 mm_ISO160_1-15 s_IncandescentWB
795 insects counted in captured image.
729 images after non-maximal-suppression

Racour_20200713_C_F10_51 mm_ISO160_1-15 s_IncandescentWB
889 insects counted in captured image.
796 images after non-maximal-suppression

Herent_w29_3-90_F10_51 mm_ISO160_1-15 s_IncandescentWB_overlay
1834 insects counted in captured image.
1554 images after non-maximal-suppression

Herent_w29_4-60_F10_51 mm_ISO160_1-15 s_IncandescentWB
313 insects counted in captured image.
276 images after non-maximal-suppression

Herent_w29_1-60_F10_51 mm_ISO160_1-15 s_IncandescentWB
484 insects counted in captured image.
447 images after non-maximal-suppression

Herent_w29_3-60_F10_51 mm_ISO160_1-15 s_IncandescentWB
502 insects counted in captured image.
453 images after non-maximal-suppression

Herent_w29_centroid_F10_51 mm_ISO160_1-15 s_IncandescentWB
274 insects counted in captured image.
234 images after non-maximal-suppression

Herent_w29_4-30_F10_51 mm_ISO1

In [84]:
pd.concat(all_dfs).to_csv(f"{images_path}/results_{date}.xlsx", sep=',')